In [1]:
# coding=utf-8
import time
from visdom import Visdom
import requests
import os
import numpy as np

viz = Visdom(server='http://127.0.0.1', port=8097)
assert viz.check_connection()




Setting up a new session...


# 一、修改loss曲线

12月9日的作业loss曲线一次只能显示一个epoch，并不连续。请你修改代码（修改方式不限），使所有epoch的loss都连续地显示在同一个window里。把修改后的代码补充在下面。在注释中标明自己修改的内容。

In [2]:
# tips：利用各种方式提升模型准确率，如修改超参、调整网络结构、或做数据增强
# tips: 优化的目标：1、从效果上优化 2、从速度上优化
#
# code：
import visdom
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
from collections import OrderedDict
from torchvision.datasets.mnist import MNIST
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
import matplotlib.pyplot as plt
if not os.path.exists('./model/'):
    os.makedirs('./model/')
viz = visdom.Visdom()

best_acc = 0
data_train_1 = MNIST('./data/mnist',
                     download=True,
                     transform=transforms.Compose([
                         transforms.Resize((32, 32)),
                         transforms.ToTensor(),
                         #TODO
                         transforms.RandomAffine(degrees=10,translate=(0.1,0.1),scale=(0.8,1.2),shear=10,fillcolor=0)]))

data_val = MNIST('./data/mnist',
                 train=False,
                 download=True,
                 transform=transforms.Compose([
                      transforms.Resize((32, 32)),
                      transforms.ToTensor()]))
#TODO
# print(data_train_1[1])
data_train_loader = DataLoader(data_train_1,batch_size=256,shuffle=True,num_workers=8)#此处可以考虑数据增强，比如合并，旋转，平移，构造一些新的数据。不做数据增强的话就复制上一题的代码在这里。

# for i in range(10):
#     plt.imshow(data_train_1[i][0].view(32,32))
#     plt.show()
data_val_loader = DataLoader(data_val, batch_size=1024, num_workers=8)
# print(data_val_loader.size())



class optimalLeNet5(nn.Module):
    def __init__(self):
        super(optimalLeNet5, self).__init__()
        #网络结构的优化，比如加入BN层？
        #TODO
        self.convnet = nn.Sequential(OrderedDict([
            ('bn0',nn.BatchNorm2d(1)),
            ('c1', nn.Conv2d(1, 6, kernel_size=(5, 5))),
            ('bn1',nn.BatchNorm2d(6)),
            ('relu1', nn.ReLU()),
            ('s2', nn.MaxPool2d(kernel_size=(2, 2), stride=2)),
            ('c3', nn.Conv2d(6, 16, kernel_size=(5, 5))),
            ('bn3',nn.BatchNorm2d(16)),
            ('relu3', nn.ReLU()),
            ('s4', nn.MaxPool2d(kernel_size=(2, 2), stride=2)),
            ('c5', nn.Conv2d(16, 120, kernel_size=(5, 5))),
            ('bn5',nn.BatchNorm2d(120)),
            ('relu5', nn.ReLU())
        ]))

        self.fc = nn.Sequential(OrderedDict([
            # ('bn5',nn.BatchNorm1d(120)),
            ('f6', nn.Linear(120, 84)),
            # ('bn6',nn.BatchNorm1d(84)),
            ('relu6', nn.ReLU()),
            ('f7', nn.Linear(84, 10)),
            ('sig7', nn.LogSoftmax(dim=-1))
        ]))
    def forward(self, img):
        #TODO
        output = self.convnet(img)
        output = output.view(img.size(0), -1)
        output = self.fc(output)
        return output
def init_parameters(n):
    if isinstance(n, nn.Linear):
        nn.init.xavier_uniform_(n.weight)

        
        
net = optimalLeNet5().cuda()
net.apply(init_parameters)
criterion = nn.CrossEntropyLoss()
#TODO
optimizer = optim.Adam(net.parameters(), lr=0.01)#尝试修改learningRate？换个optimizer？
# optimizer=optim.SGD(net.parameters(),lr=0.01)

# for visualization
train_win = None
win_opts = {
    'title': 'Epoch Loss Trace',
    'xlabel': 'Epoch Number',
    'ylabel': 'Loss',
    'width': 1200,
    'height': 600,
}
def train(epoch):
    global train_win
    net.train()
    loss_list, batch_list = [], []
    avg_loss=0.0#总loss
    total_correct=0
    for i, (images, labels) in enumerate(data_train_loader):
        images=images.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()

        output = net(images)

        loss = criterion(output, labels)
        avg_loss+=loss.detach().cpu().sum().item()
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).detach().sum().item()
        loss.backward()
        optimizer.step()
    avg_loss/=len(data_train_1)#平均loss
    print('Epoch: %d Train      Avg. Loss: %f, Accuracy: %f'%(epoch,avg_loss,total_correct/len(data_train_1)))
    if viz.check_connection():
        train_win=viz.line(Y=torch.Tensor([avg_loss]),X=torch.Tensor([epoch]),win=train_win,
                                 name='train_loss',update=None if train_win is None else 'append',
                                 opts=win_opts)#将‘replace’修改为‘append’
def validate(num):
    global best_acc,train_win
    net.eval()
    total_correct = 0
    avg_loss = 0.0
    for i, (images, labels) in enumerate(data_val_loader):
        images=images.cuda()
        labels=labels.cuda()
        output = net(images)
        avg_loss += criterion(output, labels).sum().item()
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()
        # if num==EPOCHS-1:
        #     for j,lb in enumerate(labels):
        #         if labels[j]!=pred[j]:
        #             plt.imshow(images[j].cpu().view(32,32))
        #             plt.show()
        #             print(f'ans:{labels[j].item()},pred:{pred[j].item()}')
    avg_loss /= len(data_val)
    print('Epoch: %d Validation Avg. Loss: %f, Accuracy: %f' % (num,avg_loss, float(total_correct) / len(data_val)))
    if viz.check_connection():
        train_win=viz.line(Y=torch.Tensor([avg_loss]),X=torch.Tensor([num]),win=train_win,
                                 name='test_loss',update=None if train_win is None else 'append',
                                 opts=win_opts)#将‘replace’修改为‘append’
    if float(total_correct) / len(data_val) > best_acc:
        best_acc = float(total_correct) / len(data_val)
        torch.save(net.state_dict(), './model/best_model.pt')
EPOCHS=16
def main():
    #TODO
    for epoch in range(EPOCHS):
        global data_train_1,data_train_loader
        data_train_1 = MNIST('./data/mnist',
                     download=True,
                     transform=transforms.Compose([
                         transforms.Resize((32, 32)),
                         transforms.ToTensor(),
                         transforms.RandomAffine(degrees=10,translate=(0.1,0.1),scale=(0.8,1.2),shear=10,fillcolor=0)]))
        data_train_loader = DataLoader(data_train_1,batch_size=256,shuffle=True,num_workers=8)#此处可以考虑数据增强，比如合并，旋转，平移，构造一些新的数
        train(epoch)
        validate(epoch)
main()

Setting up a new session...
Epoch: 0 Train      Avg. Loss: 0.000947, Accuracy: 0.922000
Epoch: 0 Validation Avg. Loss: 0.000056, Accuracy: 0.982800
Epoch: 1 Train      Avg. Loss: 0.000390, Accuracy: 0.969267
Epoch: 1 Validation Avg. Loss: 0.000035, Accuracy: 0.988500
Epoch: 2 Train      Avg. Loss: 0.000336, Accuracy: 0.973033
Epoch: 2 Validation Avg. Loss: 0.000034, Accuracy: 0.989800
Epoch: 3 Train      Avg. Loss: 0.000289, Accuracy: 0.976633
Epoch: 3 Validation Avg. Loss: 0.000055, Accuracy: 0.982000
Epoch: 4 Train      Avg. Loss: 0.000265, Accuracy: 0.978500
Epoch: 4 Validation Avg. Loss: 0.000038, Accuracy: 0.988300
Epoch: 5 Train      Avg. Loss: 0.000260, Accuracy: 0.979950
Epoch: 5 Validation Avg. Loss: 0.000030, Accuracy: 0.990700
Epoch: 6 Train      Avg. Loss: 0.000241, Accuracy: 0.981117
Epoch: 6 Validation Avg. Loss: 0.000025, Accuracy: 0.992800
Epoch: 7 Train      Avg. Loss: 0.000231, Accuracy: 0.981450
Epoch: 7 Validation Avg. Loss: 0.000097, Accuracy: 0.971200
Epoch: 8 Tra

# 二、visdom练习

填补以下的visdom模块并运行得到结果。
参考：https://github.com/facebookresearch/visdom

**和上一题放在同一个env里保存，把env(json文件)和ipynb文件一起提交。（内容不限，画想画的就可以）**

In [3]:

# 图片
# 单张图片
import cv2
img=cv2.imread('cat.jpg',cv2.IMREAD_COLOR)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img=np.transpose(img,(2,0,1))
print(img.shape)
# plt.imshow(img)
# plt.show()
#your code here
viz.image(
    #your code here
    img,
    opts=dict(title='不学啦！',caption='不学啦！',width=500,height=500)
)

(3, 158, 133)


'window_3920935c590602'

In [4]:

# 散点图

X=np.random.rand(100,3)
Y=np.random.randint(1,3,100)
#your code here
old_scatter = viz.scatter(
#your code here
    X=X,Y=Y,
    opts=dict(legend=['pwp','qwq'],markersize=3)
)

In [5]:

# 柱状图


#your code here
viz.bar(
#your code here
    X=np.random.rand(10)
)


#your code here
viz.bar(
#your code here
    X=np.random.rand(10,3),
    opts=dict(
        stacked=True,
        legend=['pwp','qwq','pwq'],
        rownames=list(range(2011,2021))
    )
)

'window_3920935c60b0f0'

In [6]:
# 热力图

#your code here
viz.heatmap(
#your code here
    X=np.outer(np.arange(1,50),np.arange(1,50))
    # X=data_train_1[0][0].view(32,32)
)

'window_3920935c64d372'

In [7]:
# 地表图
x=np.random.rand(50)
y=np.random.rand(50)
for i in range(1,50):
    x[i]+=x[i-1]
    y[i]+=y[i-1]
#your code here
viz.contour(
#your code here
    X=np.outer(x,y)
)

'window_3920935c68b64a'

In [9]:


# 表面图
x=cv2.getGaussianKernel(51,6)
y=cv2.getGaussianKernel(51,9)
x=x@x.T
y=y@y.T
#your code here
viz.surf(
    #your code here
    X=x-y,
    opts=dict(title='Difference of Gaussian')
)

'window_3920936e7bf35e'

# 三、（选做）可视化：Class Activation Map(CAM)
运行CAM或grad-CAM的代码并展示效果。数据图片不限，但记得要使用分类任务的数据。

不要求完全自己实现，推荐使用开源实现（也可以不用pytorch）。请用注释标注计算热力图权重的关键模块，并写出自己对关键模块的理解。




In [9]:
#Code Here







